In [1]:
# ====================================
# 1. IMPORTY I USTAWIENIA
# ====================================
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

print(tf.__version__)


2.18.0


In [2]:
# ====================================
# 2. HYPERPARAMETRY
# ====================================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 6  # baseball, cricket, mushing, nascar racing, rock climbing, swimming
EPOCHS = 10

In [4]:
# ====================================
# 3. TWORZENIE GENERATORÓW
# ====================================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory='data/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    directory='data/valid',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory='data/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 901 images belonging to 6 classes.
Found 30 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [5]:
# ====================================
# 4. BUDOWANIE MODELU (TRANSFER LEARNING z MobileNetV2)
# ====================================
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Zamrażamy wagi bazowego modelu
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# ====================================
# 5. TRENING
# ====================================
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator
)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step - accuracy: 0.5695 - loss: 1.2042

C:\Users\barte\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 336ms/step - accuracy: 0.5756 - loss: 1.1869 - val_accuracy: 0.9667 - val_loss: 0.1225
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 215ms/step - accuracy: 0.9351 - loss: 0.2000 - val_accuracy: 0.9667 - val_loss: 0.0624
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 209ms/step - accuracy: 0.9270 - loss: 0.1808 - val_accuracy: 0.9667 - val_loss: 0.0712
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 210ms/step - accuracy: 0.9350 - loss: 0.1856 - val_accuracy: 1.0000 - val_loss: 0.0254
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 213ms/step - accuracy: 0.9582 - loss: 0.1185 - val_accuracy: 1.0000 - val_loss: 0.0141
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 209ms/step - accuracy: 0.9664 - loss: 0.1151 - val_accuracy: 1.0000 - val_loss: 0.0090
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 215ms/step - accuracy: 0.9582 - loss: 0.1014 - val_accuracy: 1.0000 - val_loss: 0.0145
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 211ms/step - accuracy: 0.9472 - loss: 0.1170 - val_accuracy: 1.0000 - val

In [8]:
# ====================================
# 6. EWALUACJA
# ====================================
val_loss, val_accuracy = model.evaluate(valid_generator)
print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}")

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 1.0000 - loss: 0.0036
Validation loss: 0.0036, Validation accuracy: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.8667 - loss: 0.2419
Test loss: 0.2419, Test accuracy: 0.8667


In [9]:
# ====================================
# 7. ZAPISANIE MODELU
# ====================================
model.save("sport_classifier.keras")

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Wczytujemy nasz zapisany model
model = load_model("sport_classifier.keras")

# Nazwy klas w takiej kolejności, w jakiej były trenowane
# Upewnij się, że indeksy pokrywają się z kolejnością w generatorze
class_names = [
    "baseball",
    "cricket",
    "mushing",
    "nascar racing",
    "rock climbing",
    "swimming"
]


In [23]:
# Funkcja do klasyfikacji pojedynczego obrazu
def predict_sport(model, image_path):
    # 1. Wczytaj i przeskaluj obraz (224x224)
    img = load_img(image_path, target_size=(224, 224))

    # 2. Przekształć w tablicę NumPy i znormalizuj wartości [0,1]
    img_array = img_to_array(img) / 255.0

    # 3. Dodaj wymiar batch (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)

    # 4. Przewidywanie
    predictions = model.predict(img_array)

    # 5. Wybór klasy z najwyższym prawdopodobieństwem
    predicted_index = np.argmax(predictions[0])
    predicted_label = class_names[predicted_index]
    confidence = predictions[0][predicted_index]  # wartość pewności

    return predicted_label, confidence

# Przykładowe użycie
image_path = "image10.jpg"  # Podmień na ścieżkę do własnego zdjęcia
sport_label, conf = predict_sport(model, image_path)

print(f"Przewidywany sport: {sport_label}, pewność: {conf*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Przewidywany sport: cricket, pewność: 99.45%
